In [2]:
import pandas as pd
import os

iterations=[1, 2, 3]
delays=[0.016, 0.025, 0.05, 0.1, 0.2]

raw_path = "../controlled_DATA/multiple-rates-driversapps/results_inputdriver/"

# Get the start and end files and create another file with the latency in milliseconds

for delay in delays:

    for i in iterations:

        df_start = None
        df_end = None
        df = None

        df_start = pd.read_csv(raw_path+"batch"+str(delay)+"_"+str(i)+"-start.txt", header=None, names=["hash", "nano-start"])

        df_end = pd.read_csv(raw_path+"batch"+str(delay)+"_"+str(i)+"-end.txt", header=None, names=["hash", "nano-end"])


        ### MERGE DATAFRAMES id_start and id_end
        df = df_start.merge(df_end, left_on=['hash'], right_on=['hash'], how='outer')
        if not df.isnull().sum().sum() == 0:
            print("some hash do not coincide")
        df.dropna(inplace=True)

        df["processing_time"] = df["nano-end"].astype(float) - df["nano-start"].astype(float)
        df["processing_time_ms"] = df["processing_time"]/1000000
        if not os.path.exists(raw_path):
            os.makedirs(raw_path)
        df.to_csv(raw_path+"processed/latency"+str(delay)+"_"+str(i)+".csv", index=False)

df.head()

some hash do not coincide


OSError: Cannot save file into a non-existent directory: '..\controlled_DATA\multiple-rates-driversapps\results_inputdriver\processed'

In [ ]:
delays=[0.016, 0.025, 0.05, 0.1, 0.2]

processed_path = raw_path+"processed/"

for delay in delays:

    df1 = pd.read_csv(processed_path+"latency"+str(delay)+"_"+str(1)+".csv")
    df2 = pd.read_csv(processed_path+"latency"+str(delay)+"_"+str(2)+".csv")
    df3 = pd.read_csv(processed_path+"latency"+str(delay)+"_"+str(3)+".csv")

    df = df1.merge(df2[["hash", "processing_time_ms"]], left_on=['hash'], right_on=['hash'], how='outer')
    df = df.merge(df3[["hash", "processing_time_ms"]], left_on=['hash'], right_on=['hash'], how='outer')
    if not df.isnull().sum().sum() == 0:
        print("some hash do not coincide")
    df.dropna(inplace=True)

    df["processing_time_ms"] = (df["processing_time_ms"]+df["processing_time_ms_x"]+df["processing_time_ms_y"])/3
    df.drop(['nano-start', 'nano-end', 'processing_time', 'processing_time_ms_x', 'processing_time_ms_y'], axis='columns', inplace=True)

    if len(df1) != 1250 or len(df2) != 1250 or len(df3) != 1250:
        print("Something may be wrong with this batches")

    if not os.path.exists(processed_path):
        os.makedirs(processed_path)
    df.to_csv(processed_path+"processed_averaged/latency"+str(delay)+".csv", index=False)